In [1]:
%matplotlib qt

# std
import os
from os import path
import time

# 3p
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prj
os.chdir("..")
from src import helpers, configuration, filtering, bandpower, plots, statistics as stats
from src.configuration import cfg, cfg_mr
os.chdir('analysis-adelie')

# statistics
from scipy.stats import ttest_ind

# plots
import matplotlib

font = {'family':'DejaVu Sans', 'weight':'normal', 'size':20}
matplotlib.rc('font', **font)

In [3]:
def get_concat_signal(data_type, subject="adelie", config=cfg):
    dfs = [
        helpers.load_signal_data(data_type, subject=subject, recording=recording, config=config)
        for recording in range(
            len(config["paths"]["subjects"][subject]["recordings"][data_type])
        )
    ]

    for i in range(len(dfs) - 1):
        dfs[i + 1].index += dfs[i].index.max()

    concatenated = pd.concat(dfs)
    return concatenated

## Matthieu Ricard: eigenvalues and PCA

In [6]:
baseline_mr = get_concat_signal('baseline', subject='mr', config=cfg_mr)
meditation_mr = get_concat_signal('meditation', subject='mr', config=cfg_mr)

In [7]:
# get information MR 
baseline_mr_information = stats.get_information(baseline_mr)
meditation_mr_information = stats.get_information(meditation_mr)

In [21]:
okkk = helpers.loqd_signal_data('baseline', subject='mr', config=cfg_mr, mr=True).groupby(pd.Grouper(freq='10s')).corr().groupby(level=0).filter(
    lambda g: g.isnull().sum().sum() == 0
)

AttributeError: module 'src.helpers' has no attribute 'loqd_signal_data'

In [ ]:
okkk.groupby(level=0).apply(
    lambda x: sum(np.linalg.eig(x)[0][:5])
)

In [15]:
# ttest over all electrodes
concat_mr_information = pd.concat([baseline_mr_information, meditation_mr_information], keys=['baseline', 'meditation'])
ttest_mr_information = concat_mr_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [16]:
ttest_mr_information

,
,time


In [10]:
# plot
concat_mr_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Matthieu Ricard")
plt.show()

DataError: No numeric types to aggregate

### 2nd metric: normalized sum of the five highest eigenvalues

In [234]:
# get normalized information MR 
baseline_mr_norm_information = stats.get_norm_information(baseline_mr)
meditation_mr_norm_information = stats.get_norm_information(meditation_mr)

In [235]:
# ttest over all electrodes
concat_mr_norm_information = pd.concat([baseline_mr_norm_information, meditation_mr_norm_information], keys=['baseline', 'meditation'])
ttest_mr_norm_information = concat_mr_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [236]:
ttest_mr_norm_information

Ttest_indResult(statistic=-0.30476290676964884, pvalue=0.7620173308167885)

In [270]:
# plot
concat_mr_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Matthieu Ricard")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Adélie')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [238]:
# get normalized distribution MR and mean of ev over all epochs
baseline_mr_distribution_mean = stats.get_distrib_information(baseline_mr).mean()
meditation_mr_distribution_mean = stats.get_distrib_information(meditation_mr).mean()

In [271]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_mr_distribution_mean
y2 = meditation_mr_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Matthieu Ricard')
plt.legend()
plt.show()

## Adélie: eigenvalues and PCA

In [229]:
baseline_adelie = get_concat_signal('baseline', subject='adelie')
meditation_adelie = get_concat_signal('meditation', subject='adelie')

### 1st metric: sum of the five highest eigenvalues

In [230]:
# get information Adélie 
baseline_adelie_information = stats.get_information(baseline_adelie)
meditation_adelie_information = stats.get_information(meditation_adelie)

In [231]:
# ttest over all electrodes
concat_adelie_information = pd.concat([baseline_adelie_information, meditation_adelie_information], keys=['baseline', 'meditation'])
ttest_adelie_information = concat_adelie_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [232]:
ttest_adelie_information

Ttest_indResult(statistic=-0.30476290676964746, pvalue=0.7620173308167895)

In [244]:
# plot
concat_adelie_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Adélie")
plt.show()

### 2nd metric: normalized sum of the five highest eigenvalues

In [234]:
# get normalized information Adélie 
baseline_adelie_norm_information = stats.get_norm_information(baseline_adelie)
meditation_adelie_norm_information = stats.get_norm_information(meditation_adelie)

In [235]:
# ttest over all electrodes
concat_adelie_norm_information = pd.concat([baseline_adelie_norm_information, meditation_adelie_norm_information], keys=['baseline', 'meditation'])
ttest_adelie_norm_information = concat_adelie_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [236]:
ttest_adelie_norm_information

Ttest_indResult(statistic=-0.30476290676964884, pvalue=0.7620173308167885)

In [270]:
# plot
concat_adelie_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Adélie")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Adélie')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [238]:
# get normalized distribution Adélie and mean of ev over all epochs
baseline_adelie_distribution_mean = stats.get_distrib_information(baseline_adelie).mean()
meditation_adelie_distribution_mean = stats.get_distrib_information(meditation_adelie).mean()

In [239]:
# bar plot, not very clean
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']
barWidth = 0.4

y1 = baseline_adelie_distribution_mean
y2 = meditation_adelie_distribution_mean
r1 = range(len(y1))
r2 = [x + barWidth for x in r1]

b1 = plt.bar(r1, y1, width = barWidth)
b2 = plt.bar(r2, y2, width = barWidth)

plt.xticks([r + barWidth / 2 for r in range(len(y1))], ev)

plt.title('Spectrum of normalized eigenvalues for Adélie')
plt.legend([b1, b2],['baseline', 'meditation'])

In [271]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_adelie_distribution_mean
y2 = meditation_adelie_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Adélie')
plt.legend()
plt.show()

## Sam: eigenvalues and PCA

In [247]:
baseline_sam = get_concat_signal('baseline', subject='sam')
meditation_sam = get_concat_signal('meditation', subject='sam')

### 1st metric: sum of the five highest eigenvalues

In [248]:
# get information Sam
baseline_sam_information = stats.get_information(baseline_sam)
meditation_sam_information = stats.get_information(meditation_sam)

In [249]:
# ttest over all electrodes
concat_sam_information = pd.concat([baseline_sam_information, meditation_sam_information], keys=['baseline', 'meditation'])
ttest_sam_information = concat_sam_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [250]:
ttest_sam_information

Ttest_indResult(statistic=2.8712933388578974, pvalue=0.005901815227511391)

In [272]:
# plot
%matplotlib qt
concat_sam_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Sam")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Sam')

### 2nd metric: normalized sum of the five highest eigenvalues

In [252]:
# get normalized information Sam
baseline_sam_norm_information = stats.get_norm_information(baseline_sam)
meditation_sam_norm_information = stats.get_norm_information(meditation_sam)

In [253]:
# ttest over all electrodes
concat_sam_norm_information = pd.concat([baseline_sam_norm_information, meditation_sam_norm_information], keys=['baseline', 'meditation'])
ttest_sam_norm_information = concat_sam_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [254]:
ttest_sam_norm_information

Ttest_indResult(statistic=2.871293338857912, pvalue=0.005901815227511152)

In [273]:
# plot
concat_sam_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Sam")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Sam')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [256]:
# get normalized distribution Sam and mean of ev over all epochs
baseline_sam_distribution_mean = stats.get_distrib_information(baseline_sam).mean()
meditation_sam_distribution_mean = stats.get_distrib_information(meditation_sam).mean()

In [257]:
# bar plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']
barWidth = 0.4

y1 = baseline_sam_distribution_mean
y2 = meditation_sam_distribution_mean
r1 = range(len(y1))
r2 = [x + barWidth for x in r1]

b1 = plt.bar(r1, y1, width = barWidth)
b2 = plt.bar(r2, y2, width = barWidth)

plt.xticks([r + barWidth / 2 for r in range(len(y1))], ev)

plt.title('Spectrum of normalized eigenvalues for Sam')
plt.legend([b1, b2],['baseline', 'meditation'])

In [259]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_sam_distribution_mean
y2 = meditation_sam_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Sam')
plt.legend()
plt.show()

## Sam2: eigenvalues and PCA

In [260]:
baseline_sam2 = get_concat_signal('baseline', subject='sam2')
meditation_sam2 = get_concat_signal('meditation', subject='sam2')

### 1st metric: sum of the five highest eigenvalues

In [261]:
# get information Sam2
baseline_sam2_information = stats.get_information(baseline_sam2)
meditation_sam2_information = stats.get_information(meditation_sam2)

In [276]:
# ttest over all electrodes
concat_sam2_information = pd.concat([baseline_sam2_information, meditation_sam2_information], keys=['baseline', 'meditation'])
ttest_sam2_information = concat_sam2_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [277]:
ttest_sam2_information

Ttest_indResult(statistic=-2.297931730451149, pvalue=0.025935613349770705)

In [278]:
# plot
concat_sam2_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Sam2")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Sam2')

### 2nd metric: normalized sum of the five highest eigenvalues

In [266]:
# get normalized information Sam2
baseline_sam2_norm_information = stats.get_norm_information(baseline_sam2)
meditation_sam2_norm_information = stats.get_norm_information(meditation_sam2)

In [267]:
# ttest over all electrodes
concat_sam2_norm_information = pd.concat([baseline_sam2_norm_information, meditation_sam2_norm_information], keys=['baseline', 'meditation'])
ttest_sam2_norm_information = concat_sam2_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [268]:
ttest_sam2_norm_information

Ttest_indResult(statistic=-2.2979317304511975, pvalue=0.025935613349767704)

In [275]:
# plot
concat_sam2_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Sam2")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Sam2')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [279]:
# get normalized distribution Sam2
baseline_sam2_distribution = stats.get_distrib_information(baseline_sam2)
meditation_sam2_distribution = stats.get_distrib_information(meditation_sam2)

In [280]:
# mean of ev over all epochs
baseline_sam2_distribution_mean = baseline_sam2_distribution.mean()
meditation_sam2_distribution_mean = meditation_sam2_distribution.mean()

In [281]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_sam2_distribution_mean
y2 = meditation_sam2_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Sam2')
plt.legend()
plt.show()

## Sam3: eigenvalues and PCA

In [282]:
baseline_sam3 = get_concat_signal('baseline', subject='sam3')
meditation_sam3 = get_concat_signal('meditation', subject='sam3')

### 1st metric: sum of the five highest eigenvalues

In [283]:
# get information Sam3
baseline_sam3_information = stats.get_information(baseline_sam3)
meditation_sam3_information = stats.get_information(meditation_sam3)

In [284]:
# ttest over all electrodes
concat_sam3_information = pd.concat([baseline_sam3_information, meditation_sam3_information], keys=['baseline', 'meditation'])
ttest_sam3_information = concat_sam3_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [285]:
ttest_sam3_information

Ttest_indResult(statistic=4.377924869013054, pvalue=0.0007119019407478107)

In [286]:
# plot
concat_sam3_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Sam3")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Sam3')

### 2nd metric: normalized sum of the five highest eigenvalues

In [287]:
# get normalized information Sam3
baseline_sam3_norm_information = stats.get_norm_information(baseline_sam3)
meditation_sam3_norm_information = stats.get_norm_information(meditation_sam3)

In [288]:
# ttest over all electrodes
concat_sam3_norm_information = pd.concat([baseline_sam3_norm_information, meditation_sam3_norm_information], keys=['baseline', 'meditation'])
ttest_sam3_norm_information = concat_sam3_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [289]:
ttest_sam3_norm_information

Ttest_indResult(statistic=4.377924869013023, pvalue=0.0007119019407478558)

In [290]:
# plot
concat_sam3_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Sam3")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Sam3')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [291]:
# get normalized distribution Sam3
baseline_sam3_distribution = stats.get_distrib_information(baseline_sam3)
meditation_sam3_distribution = stats.get_distrib_information(meditation_sam3)

In [292]:
# mean of ev over all epochs
baseline_sam3_distribution_mean = baseline_sam3_distribution.mean()
meditation_sam3_distribution_mean = meditation_sam3_distribution.mean()

In [293]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_sam3_distribution_mean
y2 = meditation_sam3_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Sam3')
plt.legend()
plt.show()

## Sam4: eigenvalues and PCA

In [294]:
baseline_sam4 = get_concat_signal('baseline', subject='sam4')
meditation_sam4 = get_concat_signal('meditation', subject='sam4')

### 1st metric: sum of the five highest eigenvalues

In [295]:
# get information Sam4
baseline_sam4_information = stats.get_information(baseline_sam4)
meditation_sam4_information = stats.get_information(meditation_sam4)

In [296]:
# ttest over all electrodes
concat_sam4_information = pd.concat([baseline_sam4_information, meditation_sam4_information], keys=['baseline', 'meditation'])
ttest_sam4_information = concat_sam4_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [297]:
ttest_sam4_information

Ttest_indResult(statistic=2.716447294774889, pvalue=0.018414358741238788)

In [298]:
# plot
concat_sam4_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Sam4")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Sam4')

### 2nd metric: normalized sum of the five highest eigenvalues

In [299]:
# get normalized information Sam4
baseline_sam4_norm_information = stats.get_norm_information(baseline_sam4)
meditation_sam4_norm_information = stats.get_norm_information(meditation_sam4)

In [300]:
# ttest over all electrodes
concat_sam4_norm_information = pd.concat([baseline_sam4_norm_information, meditation_sam4_norm_information], keys=['baseline', 'meditation'])
ttest_sam4_norm_information = concat_sam4_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [301]:
ttest_sam4_norm_information

Ttest_indResult(statistic=2.7164472947748814, pvalue=0.01841435874123904)

In [302]:
# plot
concat_sam4_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Sam4")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Sam4')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [303]:
# get normalized distribution Sam4
baseline_sam4_distribution = stats.get_distrib_information(baseline_sam4)
meditation_sam4_distribution = stats.get_distrib_information(meditation_sam4)

In [304]:
# mean of ev over all epochs
baseline_sam4_distribution_mean = baseline_sam4_distribution.mean()
meditation_sam4_distribution_mean = meditation_sam4_distribution.mean()

In [305]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_sam4_distribution_mean
y2 = meditation_sam4_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Sam4')
plt.legend()
plt.show()

## Raphaël: eigenvalues and PCA

In [306]:
baseline_rap = get_concat_signal('baseline', subject='raphael')
meditation_rap = get_concat_signal('meditation', subject='raphael')

### 1st metric: sum of the five highest eigenvalues

In [307]:
# get information Raphaël
baseline_rap_information = stats.get_information(baseline_rap)
meditation_rap_information = stats.get_information(meditation_rap)

In [308]:
# ttest over all electrodes
concat_rap_information = pd.concat([baseline_rap_information, meditation_rap_information], keys=['baseline', 'meditation'])
ttest_rap_information = concat_rap_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [309]:
ttest_rap_information

Ttest_indResult(statistic=7.155980982334779, pvalue=1.3455897049562867e-08)

In [310]:
# plot
concat_rap_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Raphaël")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Raphaël')

### 2nd metric: normalized sum of the five highest eigenvalues

In [311]:
# get normalized information Raphaël
baseline_rap_norm_information = stats.get_norm_information(baseline_rap)
meditation_rap_norm_information = stats.get_norm_information(meditation_rap)

In [312]:
# ttest over all electrodes
concat_rap_norm_information = pd.concat([baseline_rap_norm_information, meditation_rap_norm_information], keys=['baseline', 'meditation'])
ttest_rap_norm_information = concat_rap_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [313]:
ttest_rap_norm_information

Ttest_indResult(statistic=7.155980982334761, pvalue=1.3455897049563906e-08)

In [314]:
# plot
concat_rap_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Raphaël")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Raphaël')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [315]:
# get normalized distribution Raphaël
baseline_rap_distribution = stats.get_distrib_information(baseline_rap)
meditation_rap_distribution = stats.get_distrib_information(meditation_rap)

In [316]:
df_split_baseline = pd.DataFrame(baseline_rap_distribution.to_list(), columns= ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20'])
df_split_meditation = pd.DataFrame(meditation_rap_distribution.to_list(), columns= ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20'])

In [317]:
t_test_distribution = ttest_ind(df_split_baseline, df_split_meditation, equal_var=False, nan_policy='omit')
t_test_distribution

Ttest_indResult(statistic=array([ 5.65336397, -3.97966648, -4.08600245, -5.53472441, -6.66078783,
       -6.33118764, -6.67126641, -6.42463588, -6.89023948, -6.66680943,
       -6.10165512, -3.07541892, -3.468081  , -4.66181592, -6.12823335,
       -5.76808452, -6.47682845, -6.44039561, -7.19464862, -7.13772043]), pvalue=array([1.00302600e-05, 6.23409048e-04, 4.23135005e-04, 3.25150398e-06,
       1.34608840e-07, 4.68398357e-07, 1.07855135e-07, 1.40381479e-07,
       3.17473464e-08, 4.81779727e-08, 1.87113411e-07, 3.43513187e-03,
       1.10717749e-03, 2.58260962e-05, 1.57994811e-07, 8.07270350e-07,
       1.32740808e-07, 9.91666401e-08, 8.58931057e-09, 1.39628774e-08]))

In [318]:
# mean of ev over all epochs
baseline_rap_distribution_mean = baseline_rap_distribution.mean()
meditation_rap_distribution_mean = meditation_rap_distribution.mean()

In [319]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_rap_distribution_mean
y2 = meditation_rap_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Raphaël')
plt.legend()
plt.show()

## Raphaël2: eigenvalues and PCA

In [320]:
baseline_rap2 = get_concat_signal('baseline', subject='raphael2')
meditation_rap2 = get_concat_signal('meditation', subject='raphael2')

### 1st metric: sum of the five highest eigenvalues

In [321]:
# get information Raphaël2
baseline_rap2_information = stats.get_information(baseline_rap2)
meditation_rap2_information = stats.get_information(meditation_rap2)

In [322]:
# ttest over all electrodes
concat_rap2_information = pd.concat([baseline_rap2_information, meditation_rap2_information], keys=['baseline', 'meditation'])
ttest_rap2_information = concat_rap2_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [323]:
ttest_rap2_information

Ttest_indResult(statistic=2.634791050233782, pvalue=0.0140090895955259)

In [324]:
# plot
concat_rap2_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Raphaël2")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Raphaël2')

### 2nd metric: normalized sum of the five highest eigenvalues

In [325]:
# get normalized information Raphaël2
baseline_rap2_norm_information = stats.get_norm_information(baseline_rap2)
meditation_rap2_norm_information = stats.get_norm_information(meditation_rap2)

In [326]:
# ttest over all electrodes
concat_rap2_norm_information = pd.concat([baseline_rap2_norm_information, meditation_rap2_norm_information], keys=['baseline', 'meditation'])
ttest_rap2_norm_information = concat_rap2_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [327]:
ttest_rap2_norm_information

Ttest_indResult(statistic=2.634791050233787, pvalue=0.014009089595525775)

In [328]:
# plot
concat_rap2_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Raphaël2")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Raphaël2')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [329]:
# get normalized distribution Raphaël2 and mean of ev over all epochs
baseline_rap2_distribution_mean = stats.get_distrib_information(baseline_rap2).mean()
meditation_rap2_distribution_mean = stats.get_distrib_information(meditation_rap2).mean()

In [330]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_rap2_distribution_mean
y2 = meditation_rap2_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Raphaël2')
plt.legend()
plt.show()

## Raphaël3: eigenvalues and PCA

In [331]:
baseline_rap3 = get_concat_signal('baseline', subject='raphael3')
meditation_rap3 = get_concat_signal('meditation', subject='raphael3')

### 1st metric: sum of the five highest eigenvalues

In [332]:
# get information Raphaël3
baseline_rap3_information = stats.get_information(baseline_rap3)
meditation_rap3_information = stats.get_information(meditation_rap3)

In [333]:
# ttest over all electrodes
concat_rap3_information = pd.concat([baseline_rap3_information, meditation_rap3_information], keys=['baseline', 'meditation'])
ttest_rap3_information = concat_rap3_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [334]:
ttest_rap3_information

Ttest_indResult(statistic=3.0384222446553215, pvalue=0.005119853141453229)

In [336]:
# plot
concat_rap3_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Raphaël3")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Raphaël3')

### 2nd metric: normalized sum of the five highest eigenvalues

In [337]:
# get normalized information Raphaël23
baseline_rap3_norm_information = stats.get_norm_information(baseline_rap3)
meditation_rap3_norm_information = stats.get_norm_information(meditation_rap3)

In [338]:
# ttest over all electrodes
concat_rap3_norm_information = pd.concat([baseline_rap3_norm_information, meditation_rap3_norm_information], keys=['baseline', 'meditation'])
ttest_rap3_norm_information = concat_rap3_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [339]:
ttest_rap3_norm_information

Ttest_indResult(statistic=3.038422244655358, pvalue=0.005119853141452765)

In [340]:
# plot
concat_rap3_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Raphaël3")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Raphaël3')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [341]:
# get normalized distribution Raphaël2 and mean of ev over all epochs
baseline_rap3_distribution_mean = stats.get_distrib_information(baseline_rap3).mean()
meditation_rap3_distribution_mean = stats.get_distrib_information(meditation_rap3).mean()

In [342]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_rap3_distribution_mean
y2 = meditation_rap3_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Raphaël3')
plt.legend()
plt.show()

## Arnaud: eigenvalues and PCA

In [343]:
baseline_arn = get_concat_signal('baseline', subject='arnaud')
meditation_arn = get_concat_signal('meditation', subject='arnaud')

### 1st metric: sum of the five highest eigenvalues

In [344]:
# get information Arnaud
baseline_arn_information = stats.get_information(baseline_arn)
meditation_arn_information = stats.get_information(meditation_arn)

In [345]:
# ttest over all electrodes
concat_arn_information = pd.concat([baseline_arn_information, meditation_arn_information], keys=['baseline', 'meditation'])
ttest_arn_information = concat_arn_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [346]:
ttest_arn_information

Ttest_indResult(statistic=-1.5315387348777616, pvalue=0.26126866147933)

In [347]:
# plot
concat_arn_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("First metric: sum of the five highest eigenvalues / Arnaud")

Text(0.5, 1.0, 'First metric: sum of the five highest eigenvalues / Arnaud')

### 2nd metric: normalized sum of the five highest eigenvalues

In [348]:
# get normalized information Arnaud
baseline_arn_norm_information = stats.get_norm_information(baseline_arn)
meditation_arn_norm_information = stats.get_norm_information(meditation_arn)

In [349]:
# ttest over all electrodes
concat_arn_norm_information = pd.concat([baseline_arn_norm_information, meditation_arn_norm_information], keys=['baseline', 'meditation'])
ttest_arn_norm_information = concat_arn_norm_information.agg(lambda x: ttest_ind(x.xs('baseline', level=0), x.xs('meditation', level=0), equal_var=False, nan_policy='omit'))

In [350]:
ttest_arn_norm_information

Ttest_indResult(statistic=-1.5315387348777552, pvalue=0.2612686614793314)

In [351]:
# plot
concat_arn_norm_information.mean(level=0).transpose().plot.bar(rot=0, color=('C0','C1'))
plt.title("Second metric: sum of the normalized five highest eigenvalues / Arnaud")

Text(0.5, 1.0, 'Second metric: sum of the normalized five highest eigenvalues / Arnaud')

### 3rd metric: spectrum, distribution of all normalized eigenvalues

In [352]:
# get normalized distribution Arnaud
baseline_arn_distribution = stats.get_distrib_information(baseline_arn)
meditation_arn_distribution = stats.get_distrib_information(meditation_arn)

In [353]:
# mean of ev over all epochs
baseline_arn_distribution_mean = baseline_arn_distribution.mean()
meditation_arn_distribution_mean = meditation_arn_distribution.mean()

In [355]:
# plot
ev = ['ev1','ev2','ev3','ev4','ev5','ev6','ev7','ev8','ev9','ev10',
        'ev11','ev12','ev13','ev14','ev15','ev16','ev17','ev18','ev19','ev20']

y1 = baseline_arn_distribution_mean
y2 = meditation_arn_distribution_mean
r1 = range(len(ev))
r2 = range(len(ev))

b1 = plt.plot(r1, y1, label='baseline', linewidth=2.5)
b2 = plt.plot(r2, y2, label='meditation', linewidth=2.5)

plt.xticks([r for r in range(len(ev))], ev)
plt.title('Spectrum of normalized eigenvalues for Arnaud')
plt.legend()
plt.show()